In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import DataReader as pdr
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sqlalchemy import create_engine
import pymssql
from csv import DictWriter

from sklearn.linear_model import Ridge
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import itertools
import warnings
warnings.filterwarnings('ignore')

### Backtest - Linear Regression from class

In [2]:
#reads return from linear regression from first back test from class
rets_org = pd.read_csv("./rets_orig.csv", parse_dates=["date"], index_col="date")
rets_org.index = rets_org.index.to_period("M")

In [3]:
ff = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=2005)[0]/100

In [4]:
rets_org["mkt"] = ff["Mkt-RF"] + ff["RF"]
rets_org["rf"] = ff["RF"]
rets_org["ls"] = 1.3*rets_org["best"] - 0.3*rets_org["worst"]

In [5]:
#initializes result table
row = {}
criteria = ["best", "worst", "mkt", "ls"]
row["features"] = ["Linear_reg - roeq, bm, mom12m"]
xrets_org = rets_org[["best", "worst", "mkt", "ls"]].subtract(rets_org.rf, axis=0)

for col in criteria:
    row["ret_" + col] = [12*rets_org[col].mean()]
    
for col in criteria:
    row["std_" + col] = [np.sqrt(12)*rets_org[col].std()]

for col in criteria:
    row["sratio_" + col] = [np.sqrt(12)*xrets_org[col].mean()/xrets_org[col].std()]

result = pd.DataFrame(row)

In [6]:
model = Ridge(alpha=0.1)
prepoc = PolynomialFeatures(degree=2) #preprocessing, add a polynomial term of degree two to capture non-linearity
#prepoc = StandardScaler()
qt = QuantileTransformer(output_distribution="normal") #transform data to treat outliers

In [7]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
dfbase = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + dfbase
conn = create_engine(string).connect()

In [8]:
df = pd.read_sql(
    """
    select ticker, date, ret, mve, acc, agr, beta, bm, ep, gma, idiovol, lev, mom12m, mom1m, 
        operprof, roeq, roic, roaq, retvol, saleinv, currat
    from data
    where date>='2000-01'
    order by date, ticker
    """, 
    conn
)
df = df.dropna()
conn.close()

df = df.set_index(["date", "ticker"])

In [9]:
#drop largest stocks
df["size_rnk"] = df.groupby('date').mve.rank(ascending=False)
df = df[df["size_rnk"]>500] 
df.head(10)

ret        mve       acc       agr      beta        bm  \
date    ticker                                                                
2000-01 AAC     0.056338  12.072728 -0.144323 -0.139600  2.268646  1.893766   
        AACE   -0.067568  12.134491 -0.063165  0.078717  0.863053  0.339860   
        AAGP    0.364865   8.659571 -0.065554 -0.049251  3.448574  0.272639   
        AAII    0.109589  11.963973  0.040705  0.087686  1.777969  0.256150   
        AAIR    0.048276  12.603822 -0.085808 -0.132433  1.603224  0.326549   
        AAM     0.083333  10.055382  0.238524  0.252592  2.635396  1.311158   
        AAON    0.108696  11.407266 -0.012415  0.180902  0.751212  0.421501   
        AAS     0.193416  13.509400 -0.045706 -0.110461  1.048264  0.054139   
        AASP   -0.060241   8.266325  0.058921  0.526711  0.437517  0.561185   
        AATT    0.022901  12.371572 -0.174320  0.165060  2.292406  0.457054   

                      ep       gma   idiovol        lev    mom12m     mom1m  \
date    ticker                                                                
2000-01 AAC    -0.586435 -0.027672  0.084462   3.232779  0.241379 -0.013889   
        AACE    0.058082  0.201136  0.051541   0.682613  0.095833  0.125475   
        AAGP   -0.011439  0.814228  0.241297   0.040725 -0.646154 -0.195652   
        AAII    0.020018  0.451732  0.116112   0.127485 -0.510791  0.073529   
        AAIR   -0.239133  0.290907  0.104955   1.882961  0.761905 -0.020270   
        AAM    -6.091087 -0.220700  0.155609  13.180766 -0.666667 -0.294118   
        AAON    0.090306  0.530220  0.052385   0.450578  0.355705  0.138614   
        AAS     0.036317  0.240802  0.067126   1.061775 -0.619231  0.227273   
        AASP   -0.494222  0.034160  0.155298   5.795259 -0.388889  0.886364   
        AATT    0.052021  0.726804  0.088989   0.369821  0.370370  0.062162   

                 operprof      roeq      roic      roaq    retvol    saleinv  \
date    ticker                                                                 
2000-01 AAC     -0.215772  0.053986 -0.061952  0.017297  0.013900   8.178278   
        AACE     0.580319  0.049065  0.211095  0.015624  0.020628  80.949040   
        AAGP     0.876987  0.038481 -0.003277  0.032100  0.029503   4.959366   
        AAII     0.686724 -0.041963  0.078168 -0.025050  0.048943   5.162492   
        AAIR     1.032463  0.254678  0.008910  0.036292  0.035308  38.247170   
        AAM     -0.736722 -0.103489 -0.342144 -0.018583  0.055908  -0.028851   
        AAON     1.147671  0.092324  0.175195  0.047632  0.016566   8.781332   
        AAS     26.419188  0.175641  0.103378  0.012655  0.036034   9.961589   
        AASP     0.028064 -0.111601 -0.149269 -0.033838  0.225360  15.840000   
        AATT     1.570663  0.041950  0.189149  0.024038  0.011895  13.680429   

                   currat  size_rnk  
date    ticker                       
2000-01 AAC     18.508028    1629.0  
        AACE     1.066674    1583.0  
        AAGP     6.820513    3373.0  
        AAII     1.804698    1687.0  
        AAIR     0.632617    1343.0  
        AAM     19.238907    2858.0  
        AAON     2.154328    2048.0  
        AAS      1.397119     868.0  
        AASP     1.138249    3440.0  
        AATT     1.811587    1460.0

In [10]:
#transform features each month
def qt_df(d):
    x = qt.fit_transform(d)
    return pd.DataFrame(x, columns=d.columns, index=d.index)

In [11]:
#transform target each month
def qt_ser(s):
    x = s.copy()
    x = x.to_numpy().reshape(-1, 1)
    x = qt.fit_transform(x).flatten()
    return pd.Series(x, index=s.index)

In [12]:
#train and predict

def train_predict(df1, features, ridge_pipe):
    predictions = None
    dates = ["2005-01", "2010-01", "2015-01", "2020-01", "3000-01"]
    for train_date, end_date in zip(dates[:-1], dates[1:]):

        filter1 = df1.index.get_level_values("date") < train_date
        filter2 = df1.index.get_level_values("date") < end_date

        train = df1[filter1]
        test = df1[~filter1 & filter2]

        Xtrain = train[features]
        ytrain = train["target"]
        Xtest = test[features]
        ytest = test["target"]

        ridge_pipe.fit(Xtrain, ytrain)
        pred = ridge_pipe.predict(Xtest)
        pred = pd.Series(pred, index=test.index)
        predictions = pd.concat((predictions, pred))
    
    df1["predict"] = predictions
    
    return df1

In [13]:
def ranking(df2, numstocks):

    df2 = df2.dropna(subset=["predict"])

    df2["rnk"] = df2.groupby("date").predict.rank(method="first", ascending=False)
    best = df2[df2.rnk<=numstocks]

    df2["rnk"] = df2.groupby("date").predict.rank(method="first")
    worst = df2[df2.rnk<=numstocks]

    best_rets = best.groupby("date").ret.mean()
    worst_rets = worst.groupby("date").ret.mean()
    rets = pd.concat((best_rets, worst_rets), axis=1)
    rets.columns = ["best", "worst"]
       
    rets.to_csv("./rets.csv")

In [14]:
def rets_cal(ff):
    rets = pd.read_csv("./rets.csv", parse_dates=["date"], index_col="date")
    rets.index = rets.index.to_period("M")

    rets["mkt"] = ff["Mkt-RF"] + ff["RF"]
    rets["rf"] = ff["RF"]
    rets["ls"] = 1.3*rets["best"] - 0.3*rets["worst"]
    
    return rets

In [15]:
def eval1(rets, features, result, num):
    f_row = ""
    for feat in features:
        f_row += (feat + ", ")

    criteria = ["best", "worst", "mkt", "ls"]
    row = {}
    row["features"] = f_row
    xrets = rets[["best", "worst", "mkt", "ls"]].subtract(rets.rf, axis=0)
    
    for col in criteria:
        row["ret_"+col] = 12*rets[col].mean()
    
    for col in criteria:
        row["std_"+col] = np.sqrt(12)*rets[col].std()
        
    for col in criteria:
        row["sratio_"+col] = [np.sqrt(12)*xrets[col].mean()/xrets[col].std()]       
        
    row["num_feat"] = num
    
    with open('Ridge_result.csv', 'a') as f_object:
 
        # Pass the file object and a list
        # of column names to DictWriter()
        # You will get a object of DictWriter
        dictwriter_object = DictWriter(f_object, fieldnames=list(row.keys()))

        # Pass the dictionary as an argument to the Writerow()
        dictwriter_object.writerow(row)

        # Close the file object
        f_object.close()
    
    result = result.iloc[0:0]
    
    return result

In [16]:
total_features = ["acc", "agr", "beta", "bm", "ep", "gma", "idiovol", "lev", 
            "mom12m", "mom1m", "operprof", "roeq", "roic", "roaq", 
            "retvol", "saleinv", "currat"]

In [17]:
for num in range(8, len(total_features)+1):
    print(num)
    counter = 0
    for subset in itertools.combinations(total_features, num):
        features = list(subset)
        
        ridge_pipe = None
        numstocks = 200

        ridge_pipe = make_pipeline(prepoc, model) #create pipeline using SVR

        df_copy = df.copy()
        df_copy[features] = df_copy.groupby("date", group_keys=False)[features].apply(qt_df)
        df_copy["target"] = df_copy.groupby("date", group_keys=False).ret.apply(qt_ser)
        df_copy = train_predict(df_copy, features, ridge_pipe)
        ranking(df_copy, numstocks)
        rets = rets_cal(ff)
        result = eval1(rets, features, result, num)
        counter += 1
        if counter%50 == 0:
            print("Num: ", num, "Counter: ", counter)
            break
    

8
Num:  8 Counter:  50
9


In [ ]:
#original first backtest from class
(1+rets_org[["best", "worst", "mkt", "ls"]]).cumprod().plot()

In [ ]:
#SVR backtest
(1+rets[["best", "worst", "mkt", "ls"]]).cumprod().plot()

In [ ]:
#Ridge backtest (alpha=0.1)
print(12*rets[["best", "worst", "mkt", "ls"]].mean())
print(np.sqrt(12)*rets[["best", "worst", "mkt", "ls"]].std()) 

In [ ]:
#Ridge backtest (alpha=0.001)
print(12*rets[["best", "worst", "mkt", "ls"]].mean())
print(np.sqrt(12)*rets[["best", "worst", "mkt", "ls"]].std())

In [ ]:
result